In [81]:
import sys
sys.path.append("/work/multi_doc_analyzer")

import torch as T
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.cuda
from multi_doc_analyzer.structure.structure import *
# from tqdm import tqdm_notebook as tqdm

from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.tokenizers import Token, Tokenizer, WordTokenizer
from allennlp.data.token_indexers import TokenIndexer

from allennlp.data.instance import Instance
from allennlp.data.fields import TextField, LabelField, ArrayField

from multi_doc_analyzer.corpus_reader.ace2005_reader.ace05_reader import ACE05Reader

from allennlp.data.vocabulary import Vocabulary
from allennlp.data.iterators import BucketIterator, DataIterator
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder
import random

from allennlp.data.token_indexers import PretrainedBertIndexer
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders.bert_token_embedder import PretrainedBertEmbedder

In [82]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    seed=1,
    batch_size=64,
    lr=3e-4,                # learning rate
    epochs=50,
    hidden_sz=128,
    arg_sz=3,
    max_seq_len=100,
    model_folder="/work/model_checkpoint/bert_model_checkpoint/"
)

In [83]:
USE_GPU = T.cuda.is_available()
USE_GPU

True

In [84]:
# set seed for both CPU and CUDA
T.manual_seed(config.seed)

In [85]:
e_type2idx = {'X':0, 'O': 1, 'PER': 2, 'ORG': 3, 'LOC': 4, 'GPE': 5, 'FAC': 6, 'VEH': 7, 'WEA': 8}

r_label2idx = {'PHYS-lr': 1, 'PART-WHOLE-lr': 2, 'PER-SOC-lr': 3, 'ORG-AFF-lr': 4, 'ART-lr': 5, 'GEN-AFF-lr': 6,
               'PHYS-rl': 7, 'PART-WHOLE-rl': 8, 'PER-SOC-rl': 9, 'ORG-AFF-rl': 10, 'ART-rl': 11, 'GEN-AFF-rl': 12,
               'NONE': 0}


class RelationDatasetReader(DatasetReader):
    """
    Reads Structure object formatted datasets files, and creates AllenNLP instances.
    """
    def __init__(self, tokenizer: Tokenizer=None, token_indexers: Dict[str, TokenIndexer]=None, 
                 MAX_WORDPIECES: int=config.max_seq_len, 
                 is_training = False, ace05_reader: ACE05Reader=None):
        # make sure results may be reproduced when sampling...
        super().__init__(lazy=False)
        random.seed(0)
        self.is_training = is_training
        self.ace05_reader = ace05_reader
        
        # NOTE AllenNLP automatically adds [CLS] and [SEP] word peices in the begining and end of the context,
        # therefore we need to subtract 2
        self.MAX_WORDPIECES = MAX_WORDPIECES - 2
        
        self.tokenizer = tokenizer or WordTokenizer()
        
        # BERT specific init
        self._token_indexers = token_indexers

    def text_to_instance(self, sentence: Sentence) -> Instance:
#         sentence_tokens = [Token(x) for x in self.tokenizer(sentence.text)]
#         sentence_tokens = [Token(i) for t in sentence.text for i in self.tokenizer(t)]
#         sentence_tokens = [Token(text=t) for t in sentence.text]
        sentence_tokens = []
        td = self.tokenizer(sentence.text)
#         print(td)
        for t in td:
            sentence_tokens.append(Token(text=t))
        
#         for t in sentence.text:
#             td = self.tokenizer(t)
#             assert len(td) == 1 or len(td) == 0
#             if td:
#                 sentence_tokens.append(Token(text=td[0]))
#             else:
#                 sentence_tokens.append(Token(text='[MASK]'))

        sentence_field = TextField(sentence_tokens, self._token_indexers)
        fields = {"tokens": sentence_field}
#         char_list_field = ListField([t for t in sentence.text])
#         fields['char_list'] = char_list_field

        e_tuple_check_dicts = {} # {(train_arg_l.id, train_arg_r.id):true_label, ...}
        if self.is_training: 
            for r in sentence.relation_mentions:
                train_arg_l, train_arg_r, true_label = r.get_left_right_args()
                e_tuple_check_dicts[(train_arg_l.id, train_arg_r.id)] = true_label

        # construct pair entities
        for arg_left_idx in range(len(sentence.entity_mentions)-1):
            for arg_right_idx in range(arg_left_idx+1, len(sentence.entity_mentions)):
                arg_left = sentence.entity_mentions[arg_left_idx]
                arg_right = sentence.entity_mentions[arg_right_idx]
                arg_vec = T.zeros(self.MAX_WORDPIECES + 2, dtype=T.long)
                arg_vec[:len(sentence_tokens)+2] = 1

                # +1 because the first token is [CLS]
                arg_vec[arg_left.char_b+1:arg_left.char_e+1] = e_type2idx[arg_left.type]
                arg_vec[arg_right.char_b+1:arg_right.char_e+1] = e_type2idx[arg_right.type]
                fields["arg_idx"] = ArrayField(arg_vec)


#                 fields["arg_left"] = SpanField(arg_left.char_b, arg_left.char_e, char_list_field)
#                 fields["arg_right"] = SpanField(arg_right.char_b, arg_right.char_e, char_list_field)
                if self.is_training:
                    if (arg_left.id, arg_right.id) in e_tuple_check_dicts.keys():
                        fields["label"] = LabelField(r_label2idx[e_tuple_check_dicts[(arg_left.id, arg_right.id)]], skip_indexing=True)
                    else:
                        fields["label"] = LabelField(r_label2idx['NONE'], skip_indexing=True)
                yield Instance(fields)
    
    def _read(self, file_path: str)->Iterator:
        doc_dicts = self.ace05_reader.read(file_path)
        for doc in doc_dicts.values():
            for s in doc.sentences: 
                if len(s.text) <= config.max_seq_len:
                    for instance in self.text_to_instance(s):
                        yield instance

In [86]:
class BERT(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                out_sz: int=len(r_label2idx)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self._entity_embeddings = T.nn.Embedding(num_embeddings=len(e_type2idx), embedding_dim=config.arg_sz, padding_idx=0)
        self.gru = T.nn.GRU(word_embeddings.get_output_dim()+config.arg_sz, config.hidden_sz, batch_first=True)
        self.projection = nn.Linear(config.hidden_sz, out_sz)
        self.loss = nn.CrossEntropyLoss()
        
    def forward(self, tokens: Dict[str, T.tensor], arg_idx: T.tensor, label: T.tensor = None) -> Dict[str, T.tensor]:
        mask = get_text_field_mask(tokens)
        
        embeddings = self.word_embeddings(tokens)
        pad_len = embeddings.shape[-2]
        
        arg_idx = arg_idx[:,:pad_len]
        arg_idx = arg_idx.type(T.long)
        
        arg_emb = self._entity_embeddings(arg_idx)

        concat = T.cat((embeddings, arg_emb), -1)
        ot, ht = self.gru(concat, None) # revise this "None"
        ot = ot[:,-1,:]    
        class_logits = self.projection(ot)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

In [87]:
from scipy.special import expit # the sigmoid function
def tonp(tsr): return tsr.detach().cpu().numpy()

In [88]:
# Predict
class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator, total=self.iterator.get_num_batches(ds))
        preds = []
        with T.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

In [89]:

if __name__ == '__main__':
    
    ace_path = "/work/LDC2006T06/"
    models_folder = "/work/relation_extraction/bert_pretrain_model/"

    ace05_reader = ACE05Reader(lang='en')
    
    token_indexer = PretrainedBertIndexer(
        pretrained_model="bert-base-cased",
        max_pieces=config.max_seq_len,
#         truncate_long_sequences=False,
        do_lowercase=False
    )

	# AllenNLP DatasetReader
    reader = RelationDatasetReader(
        is_training=True, 
        ace05_reader=ace05_reader, 
        tokenizer=lambda s: token_indexer.wordpiece_tokenizer(s),
        token_indexers={"tokens": token_indexer}
    )

    train_ds = reader.read(ace_path)
    
    vocab = Vocabulary()
    iterator = BucketIterator(batch_size=config.batch_size, sorting_keys=[("tokens", "num_tokens")])
    iterator.index_with(vocab)

    bert_embedder = PretrainedBertEmbedder(
        pretrained_model="bert-base-cased",
        top_layer_only=True, # conserve memory   
    )
    word_embeddings: TextFieldEmbedder = BasicTextFieldEmbedder({"tokens": bert_embedder},
                                                                # we'll be ignoring masks so we'll need to set this to True
                                                               allow_unmatched_keys = True)
    model = BERT(word_embeddings)
    if USE_GPU:
        model.cuda()

    optimizer = optim.Adam(model.parameters(), lr=config.lr)


0it [00:00, ?it/s]

 93%|█████████▎| 111/119 [00:01<00:00, 100.22it/s]
21751it [00:44, 488.98it/s]0:01<00:00, 87.43it/s] 


In [90]:
    # training
    from allennlp.training.trainer import Trainer

    trainer = Trainer(
        model=model,
        optimizer=optimizer,
        iterator=iterator,
        train_dataset=train_ds,
        cuda_device=0 if USE_GPU else -1,
        num_epochs=config.epochs,
    )

In [91]:
    metrics = trainer.train()

loss: 0.5380 ||: 100%|██████████| 340/340 [00:20<00:00, 16.47it/s]
loss: 0.3851 ||: 100%|██████████| 340/340 [00:17<00:00, 19.65it/s]
loss: 0.2650 ||: 100%|██████████| 340/340 [00:18<00:00, 18.37it/s]
loss: 0.1750 ||: 100%|██████████| 340/340 [00:18<00:00, 18.66it/s]
loss: 0.1218 ||: 100%|██████████| 340/340 [00:18<00:00, 18.81it/s]
loss: 0.0916 ||: 100%|██████████| 340/340 [00:18<00:00, 18.81it/s]
loss: 0.0691 ||: 100%|██████████| 340/340 [00:19<00:00, 17.13it/s]
loss: 0.0520 ||: 100%|██████████| 340/340 [00:18<00:00, 21.26it/s]
loss: 0.0412 ||: 100%|██████████| 340/340 [00:18<00:00, 18.69it/s]
loss: 0.0332 ||: 100%|██████████| 340/340 [00:18<00:00, 18.40it/s]
loss: 0.0276 ||: 100%|██████████| 340/340 [00:18<00:00, 18.50it/s]
loss: 0.0204 ||: 100%|██████████| 340/340 [00:18<00:00, 18.21it/s]
loss: 0.0211 ||: 100%|██████████| 340/340 [00:17<00:00, 18.27it/s]
loss: 0.0161 ||: 100%|██████████| 340/340 [00:18<00:00, 20.37it/s]
loss: 0.0135 ||: 100%|██████████| 340/340 [00:17<00:00, 18.51i

In [92]:
    # save 
    with open(config.model_folder+'model.th', 'wb') as f:
        T.save(model.state_dict(), f)